In [ ]:
pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 30.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.0 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=94d14868d9ef78a053712fe9eacffa947a35345d7ee5a0bdec960813a6fe2d83
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


In [ ]:
import numpy as np
import torch.optim as optim
from tqdm import tqdm, trange
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms
np.random.seed(42)
torch.manual_seed(42)
import matplotlib.pyplot as plt
import torchvision.models as models
import wandb
from torch.optim import Adam, SGD, RMSprop

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from torchvision import datasets, transforms

# Define the transform to normalize the data and convert it to 3-channel RGB
transform = transforms.Compose([transforms.Grayscale(num_output_channels=3),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,), (0.5,))])

# Download and load the training data
trainset = datasets.FashionMNIST('~/.pytorch/FashionMNIST_data/', download=True, train=True, transform=transform)

# Download and load the test data
testset = datasets.FashionMNIST('~/.pytorch/FashionMNIST_data/', download=True, train=False, transform=transform)

# Define a function to filter the data by class
def filter_classes(dataset, classes):
    mask = torch.zeros(len(dataset.targets), dtype=bool)
    for c in classes:
        mask |= torch.tensor(dataset.targets) == c
    dataset.data = dataset.data[mask]
    dataset.targets = torch.tensor(dataset.targets)[mask]
    for i, c in enumerate(classes):
        dataset.targets[dataset.targets == c] = i
    dataset.targets = dataset.targets.tolist()

# Filter the training data by class
filter_classes(trainset, [1, 3, 5, 7, 9])

# Filter the test data by class
filter_classes(testset, [1, 3, 5, 7, 9])

# Create DataLoaders for the filtered training and test data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=True)

100%|██████████| 26421880/26421880 [00:02<00:00, 12234760.59it/s]


Extracting /root/.pytorch/FashionMNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/FashionMNIST_data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 211111.40it/s]


Extracting /root/.pytorch/FashionMNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/FashionMNIST_data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 3889685.01it/s]


Extracting /root/.pytorch/FashionMNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/FashionMNIST_data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 5773336.09it/s]


Extracting /root/.pytorch/FashionMNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/FashionMNIST_data/FashionMNIST/raw



<ipython-input-4-3c460ccc38f8>:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  mask |= torch.tensor(dataset.targets) == c
<ipython-input-4-3c460ccc38f8>:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  dataset.targets = torch.tensor(dataset.targets)[mask]


In [ ]:
# Load pre-trained ResNet18 model
resnet18 = models.resnet18(pretrained=True)
num_ftrs = resnet18.fc.in_features
resnet18.fc = nn.Linear(num_ftrs, 5)
resnet18.to(device)

# Define hyperparameters
config_defaults = {
    'learning_rate': 3e-5,
    'optimizer': 'Adam',
    'activation': 'ReLU',
}
sweep_config = {
    'method': 'random', #grid, random
    'parameters': {
        'learning_rate': {
            'values': [1e-3, 3e-4, 3e-5, 1e-5]
        },
        'optimizer': {
            'values': ['Adam', 'SGD', 'RMSprop']
        },
        'activation': {
            'values': ['ReLU', 'Tanh', 'Softmax']
        }
    }
}

# Initialize wandb
wandb.init(project='final_lab_8', config=config_defaults)
config = wandb.config
wandb.run.name = f"{config.optimizer}-lr_{config.learning_rate}-act_{config.activation}"
criterion = nn.CrossEntropyLoss()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 125MB/s]


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
sweep_id = wandb.sweep(sweep_config, project='final_lab_8')

Create sweep with ID: rxkozgms
Sweep URL: https://wandb.ai/mukul_dl/final_lab_8/sweeps/rxkozgms


In [ ]:
def train(model, device, train_loader, optimizer, epoch, activation, reg_strength=0.001):
    model.train()
    train_loss = 0
    train_correct_top1 = 0
    train_correct_top5 = 0
    num_batches = len(train_loader)
    criterion = nn.CrossEntropyLoss()
    with tqdm(total=num_batches, desc=f'Training Epoch {epoch}', unit='batch') as pbar:
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            # L2 regularization
            reg_loss = sum(param.norm(2) ** 2 for param in model.parameters())
            loss += reg_strength * reg_loss

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

            # Calculate top-1 and top-5 accuracy
            _, predicted = torch.topk(output, k=5, dim=1)
            train_correct_top1 += torch.sum(predicted[:, 0] == target)
            train_correct_top5 += torch.sum(predicted == target.unsqueeze(1))

            # Update progress bar
            pbar.set_postfix({'loss': train_loss / (batch_idx + 1)})
            pbar.update()

        # Log training metrics to WandB
        wandb.log({
            "epoch": epoch,
            "train_loss": train_loss / len(train_loader),
            "train_acc_top1": train_correct_top1 / len(train_loader.dataset)
        })

    train_loss /= len(train_loader.dataset)
    train_acc_top1 = train_correct_top1.double() / len(train_loader.dataset)
    train_acc_top5 = train_correct_top5.double() / len(train_loader.dataset)
    return train_loss, train_acc_top1, train_acc_top5

def test(model, device, test_loader, activation, reg_strength=0.001):
    model.eval()
    test_loss = 0
    test_correct = 0
    test_total = 0
    test_correct_top5 = 0
    num_batches = len(test_loader)
    criterion = nn.CrossEntropyLoss()
    with tqdm(total=num_batches, desc='Testing', unit='batch') as pbar:
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(test_loader):
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = criterion(output, target)

                test_loss += loss.item()
                _, predicted = torch.max(output.data, 1)
                test_total += target.size(0)
                test_correct += (predicted == target).sum().item()

                # Top-5 accuracy
                _, top5_predicted = torch.topk(output.data, 5, dim=1)
                for i in range(len(target)):
                    if target[i] in top5_predicted[i]:
                        test_correct_top5 += 1

                # Update progress bar
                pbar.set_postfix({'loss': test_loss / (batch_idx + 1)})
                pbar.update()

        # Log testing metrics to WandB
        wandb.log({
            "test_loss": test_loss / len(test_loader.dataset),
            "test_acc": test_correct / test_total
        })

    test_loss /= len(test_loader.dataset)
    test_acc = test_correct / test_total
    test_acc_top5 = test_correct_top5 / test_total
    return test_loss, test_acc, test_acc_top5

In [ ]:
def train_and_test():
    train_losses = []
    test_losses = []
    train_accuracies = []
    test_accuracies = []
    resnet18 = models.resnet18(pretrained=True)
    num_ftrs = resnet18.fc.in_features
    resnet18.fc = nn.Linear(num_ftrs, 5)
    resnet18.to(device)

    # Initialize wandb run
    wandb.init(project='final_lab_8', config=config_defaults)
    config = wandb.config
    wandb.run.name = f"{config.optimizer}-lr_{config.learning_rate}_{config.activation}"
    
    # Define optimizer and activation function based on hyperparameters
    if config.optimizer == 'Adam':
        optimizer = optim.Adam(resnet18.parameters(), lr=config.learning_rate)
    elif config.optimizer == 'SGD':
        optimizer = optim.SGD(resnet18.parameters(), lr=config.learning_rate, momentum=0.9)
    elif config.optimizer == 'RMSprop':
        optimizer = optim.RMSprop(resnet18.parameters(), lr=config.learning_rate)
        
    if config.activation == 'ReLU':
        activation = nn.ReLU()
    elif config.activation == 'Tanh':
        activation = nn.Tanh()
    elif config.activation == 'Softmax':
        activation = nn.Softmax(dim=1)
    
    # Define loss function
    criterion = nn.CrossEntropyLoss()
    test_losses = []
    test_accuracies_top1 = []
    test_accuracies_top5 = []
    train_losses = []
    train_accuracies_top1 = []
    train_accuracies_top5 = []
    # Train the model for 30 epochs
    for epoch in range(30):
      train_loss, train_acc_top1, train_acc_top5 = train(resnet18, device, train_loader, optimizer, epoch, activation)
      test_loss, test_acc_top1, test_acc_top5 = test(resnet18, device, test_loader, activation)
      train_losses.append(train_loss)
      train_accuracies_top1.append(train_acc_top1)
      train_accuracies_top5.append(train_acc_top5)
      test_losses.append(test_loss)
      test_accuracies_top1.append(test_acc_top1)
      test_accuracies_top5.append(test_acc_top5)
      print()
      print("Epoch: ", epoch + 1, "Train Loss: ", train_loss, "Train Acc top 1: ", train_acc_top1.item(), "Test Loss: ", test_loss, "Test Acc top 1: ", test_acc_top1)
      print("Train Acc top 5: ", train_acc_top5.item(), "Test Acc top 5: ", test_acc_top5)
      print()
        


In [ ]:
def sweep_agent():
  wandb.agent(sweep_id, function=train_and_test) 

sweep_agent()

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
wandb: Agent Starting Run: 8i2menpa with config:
wandb: 	activation: Tanh
wandb: 	learning_rate: 3e-05
wandb: 	optimizer: SGD
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 980, in _bootstrap_inner
Exception in thread NetStatThr:
    self.run()Traceback (most recent call last):
  File "/usr/lib/python3.9/threading.py", line 980, in _bootstrap_inner

  File "/usr/lib/python3.9/threading.py", line 917, in run
    self.run()
  File "/usr/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_run.py", line 277, in check_stop_status
    self._loop_check_status(
  File "/usr/local/lib/python3.9/dist-packages/wandb/sdk/wandb_run.py", line 215, in _loop_check_status
    local_handle = reques

Testing: 100%|██████████| 40/40 [00:02<00:00, 15.11batch/s, loss=0.574]



Epoch:  1 Train Loss:  0.0647190212726593 Train Acc top 1:  0.6438666666666667 Test Loss:  0.0045924136936664585 Test Acc top 1:  0.8508
Train Acc top 5:  1.0 Test Acc top 5:  1.0



Testing: 100%|██████████| 40/40 [00:02<00:00, 15.37batch/s, loss=0.355]



Epoch:  2 Train Loss:  0.06029686385790507 Train Acc top 1:  0.8771666666666668 Test Loss:  0.00283750079870224 Test Acc top 1:  0.8994
Train Acc top 5:  1.0 Test Acc top 5:  1.0



...

Testing: 100%|██████████| 40/40 [00:03<00:00, 11.33batch/s, loss=0.104]



Epoch:  14 Train Loss:  0.00025233771676818527 Train Acc top 1:  1.0 Test Loss:  0.0008288071541115642 Test Acc top 1:  0.9818
Train Acc top 5:  1.0 Test Acc top 5:  1.0



Testing: 100%|██████████| 40/40 [00:03<00:00, 11.79batch/s, loss=0.0937]



Epoch:  15 Train Loss:  0.00025110060113171735 Train Acc top 1:  1.0 Test Loss:  0.0007494253332493827 Test Acc top 1:  0.982
Train Acc top 5:  1.0 Test Acc top 5:  1.0



Training Epoch 15:  84%|████████▍ | 198/235 [00:19<00:03, 10.69batch/s, loss=0.032]

In [ ]:
def log_image_table(images, predicted, lables, probs):
  table = wandb.Table(columns = ["image", "pred", "target"]+[f"score+{i}" for i in range(10)])
  for img,pred,targ,prob in zip(images.to('cpu'), predicted.to('cpu'), lables.to('cpu'), probs.to('cpu')):
    table.add_data(wandb.Image(img[0].numpy()*255), pred, targ, *prob.numpy())
  wandb.log({'predcitions table': table}, commit = False)